# Carregamento dos dados

In [1]:
import pandas as pd

# Carregando os dados do arquivo CSV pré-processado em um DataFrame do pandas
comentarios = pd.read_csv('../Data/content_pre_processado.csv')

# Exibindo as primeiras 5 linhas do DataFrame para verificação
comentarios.head(5)

,reviewId,app_name,content,sentiment_polarity,sentiment
0,01e23c15-44bf-46b7-b280-40e880d9d49b,Shopee,sopre razoável apesar fácil utilizar apresenta...,positive,sadness
1,df40ba7c-075d-48dc-b798-21466dfb41d6,Shopee,otimizar fácil visualizar produto pedido rastr...,positive,sadness
2,59490b23-b17b-4bba-855b-b5053dadc0db,Shopee,prático fácil entender vendedor mandar produto...,positive,sadness
3,48fd8168-1e42-4475-9a99-935c2fd48769,Shopee,aplicativo chatear fato pesquisar produto colo...,positive,sadness
4,f4e1468b-6d97-4f1a-927d-1afee2bfbe75,Shopee,ter dificuldade pagamento cartão crédito compr...,negative,disgust
